<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Constantes dadas
P = 9.4573  # Presión en bar
T = 350.0   # Temperatura en K
R = 0.08314  # Constante de los gases en bar·L/mol·K

# Propiedades críticas del n-butano
Tc = 425.2  # Temperatura crítica en K
Pc = 37.96  # Presión crítica en bar
tr=T/Tc
pr=P/Pc

# Parámetros de la ecuación de Redlich-Kwong
q=(0.42748/0.08664)*(tr**(-3/2))
b=0.08664*(tr/pr)
# Definir la función de Redlich-Kwong y su derivada usando numpy
def Rk_vapor(Vm):
    zv=1+b-q*b*((zv-b)/(zv(zv+b)))
    return zv

def Rk_liquid(Vm):
    zl=b+zl*(zl+b)*((1+b-zl)/(q*b))
    return zl

# Método de Newton-Raphson
def newton_raphson(Vm_guess, tol=1e-6, max_iter=100):
    Vm = Vm_guess
    for i in range(max_iter):
        f_val = f(Vm)
        f_deriv = f_prime(Vm)

        # Evitar división por cero o valores no definidos
        if np.isclose(f_deriv,0,atol=1e-12):
           Vm +=1e-6
           continue

        Vm_new = Vm - f_val / f_deriv

        # Verificar convergencia
        if np.abs(Vm_new - Vm) < tol:
            return Vm_new

        Vm = Vm_new

    raise ValueError("El método de Newton-Raphson no convergió.")

# Estimaciones iniciales para los volúmenes molares
Vm_guess_vapor = 2.555  # Para el vapor
Vm_guess_liquid = 0.1333  # Para el líquido (valor muy cercano a b)

# Calcular los volúmenes molares
Vm_vapor = newton_raphson(Vm_guess_vapor)
Vm_liquid = newton_raphson(Vm_guess_liquid,max_iter=200)
Vm_vapor_cm3 = Vm_vapor * 1000
Vm_liquid_cm3 = Vm_liquid * 1000

# Imprimir los resultados
print("Volumen molar del vapor:", Vm_vapor_cm3)
print("Volumen molar del líquido:", Vm_liquid_cm3)


ValueError: El método de Newton-Raphson no convergió.

In [15]:
import numpy as np

# Constantes dadas
P = 9.4573  # Presión en bar
T = 350.0   # Temperatura en K
R = 0.08314  # Constante de los gases en bar·L/mol·K

# Propiedades críticas del n-butane
Tc = 425.2  # Temperatura crítica en K
Pc = 37.96  # Presión crítica en bar
tr = T / Tc  # Temperatura reducida
pr = P / Pc  # Presión reducida

# Parámetros de la ecuación de Redlich-Kwong
q = (0.42748 / 0.08664) * (tr**(-3/2))
b = 0.08664 * (tr / pr)

# Función para Z_vapor usando Redlich-Kwong
def Rk_vapor(Z):
    return Z - (1 + b - q * b * ((Z - b) / (Z * (Z + b))))

# Derivada de la función Z_vapor
def dRk_vapor(Z):
    numerator = q * b * (b * Z + 2 * Z**2)
    denominator = Z**2 * (Z + b)**2
    return 1 - numerator / denominator

# Función para Z_liquid usando Redlich-Kwong
def Rk_liquid(Z):
    return Z - (b + Z * (Z + b) * ((1 + b - Z) / (q * b)))

# Derivada de la función Z_liquid
def dRk_liquid(Z):
    term1 = 2 * Z + b
    term2 = (1 + b - Z) / (q * b)
    term3 = Z * (Z + b) / (q * b)
    return 1 - term1 * term2 + term3

# Método de Newton-Raphson
def newton_raphson(func, dfunc, Z_guess, tol=1e-3, max_iter=200):
    Z = Z_guess
    for i in range(max_iter):
        f_val = func(Z)
        f_deriv = dfunc(Z)
        if f_deriv == 0:
            raise ValueError("La derivada es cero. El método de Newton-Raphson falló.")
        Z_new = Z - f_val / f_deriv



        if np.abs(Z_new - Z) < tol:
            return Z_new
        Z = Z_new
    raise ValueError("El método de Newton-Raphson no convergió.")

# Estimaciones iniciales para los factores de compresibilidad Z
Z_guess_vapor = 0.9  # Aproximación para el vapor
Z_guess_liquid = 0.05  # Aproximación para el líquido

# Calcular Z usando Newton-Raphson
Z_vapor = newton_raphson(Rk_vapor, dRk_vapor, Z_guess_vapor)
Z_liquid = newton_raphson(Rk_liquid, dRk_liquid, Z_guess_liquid)

# Convertir Z a volúmenes molares Vm (en L/mol) y luego a cm^3/mol
Vm_vapor_cm3 = Z_vapor * R * T / P * 1000
Vm_liquid_cm3 = Z_liquid * R * T / P * 1000

Z_vapor, Z_liquid, Vm_vapor_cm3, Vm_liquid_cm3



(-0.28493792512158905,
 0.42330963504710306,
 -876.7204892636503,
 1302.4739693396268)